<div style="font-size: 200%; font-weight: bold; color: maroon;">Procesado de DataFrames de Spark con sparklyr <span style="color: green;">[solución]</span></div></div>
<!-- v. 1.2 - Paulo Villegas, 2020 -->

Este notebook es una prueba con [sparklyr](http://spark.rstudio.com/) 

# Inicialización

Carga de la librería y creadión de la `SparkSession`

In [ ]:
# SPARK_HOME needs to be defined, or else sparklyr won't be able to launch a local instance
Sys.getenv('SPARK_HOME')

In [ ]:
# Load sparklyr. And also dplyr, which plays nicely with sparklyr
library(sparklyr)

library(dplyr)

In [ ]:
# Get a connection to a Spark local instance
sc <- spark_connect(master = "local")

In [ ]:
spark_version(sc)

# Lectura de datos

## Desde CSV

Leemos un [CSV](https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.sql.DataFrameReader)  para crear un `SparkDataFrame`

In [ ]:
uni2  <- spark_read_csv( sc, "uni_table", "../../DATA/situacion-laboral-titulados-2019.csv", 
                         header=TRUE, options=list(comment="#") )

--------------------------------------------------

# Ejercicio

## Probabilidad de inactividad

<span style="color: red;">Pregunta:</span> *Si soy hombre y he estudiado en una universidad pública ¿en qué rama de conocimiento tengo más probabilidad de estar inactivo?*

Nota: calcularlo para todas las filas e inspeccionar visualmente es trampa. Tiene que ser un procedimiento que permita extraer ese dato programáticamente

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Escriba el código necesario en la siguiente celda</div>

<div class="alert alert-block alert-info">
Vamos a resolvarlo paso a paso, generando los DataFrames intermedios
</div>

In [ ]:
# Alumnos de universidades públicas que son hombres
uni.ph = filter( uni2, Sexo == "hombres", Tipo == "públicas" )

In [ ]:
# Porcentaje de esos alumnos que están inactivos
uni.ph.in = mutate( uni.ph, frac_in = Inactivo/Total )

In [ ]:
# Obtiene el máximo
# Para ello ordena por valor decreciente de fracción (usando desc) y luego aplica min_rank <= 1
in.max = filter( uni.ph.in, min_rank(desc(frac_in)) <= 1 )

# Nota: top_n es más inmediato, 
#    in.max = top_n( uni.ph.in, 1, frac_in )
# pero tiene problemas con ciertas versiones de Spark, ver https://github.com/sparklyr/sparklyr/issues/2084

In [ ]:
# Recupera desde Spark
result = collect(in.max)

In [ ]:
# Esta es la fila resultado
result

In [ ]:
# En otras palabras, la rama con el porcentaje más alto de inactivos (suponiendo hombres & univ. pública) es ...
result$Rama

<div class="alert alert-block alert-info">Ahora repetimos el proceso, pero todo de una vez, usando <em>pipes</em></div>

In [ ]:
result2 <- uni2 %>% filter( Sexo == "hombres", Tipo == "públicas" ) %>%
                    mutate( frac_in = Inactivo/Total ) %>%
                    filter( min_rank(desc(frac_in)) <= 1 ) %>%
                    select( Rama ) %>%
                    collect()

In [ ]:
result2

<div class="alert alert-block alert-info">Frecuentemente dplyr ofrece más de una forma de obtener los resultados. Por ejemplo, esta otra versión usa <code>arrrange</code> y <code>filter</code> con <code>row_number()</code></div>

In [ ]:
result3 <- uni2 %>% filter( Sexo == "hombres", Tipo == "públicas" ) %>%
                    mutate( frac_in = Inactivo/Total ) %>%
                    arrange( desc(frac_in)) %>%
                    filter( row_number() <= 1 ) %>%
                    select( Rama ) %>%
                    collect()

In [ ]:
result3